In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Rectangle
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

#### get results of sr13 by mgra

In [ ]:
sr13_sql = '''
select x.mgra, hs, increment, city, cpa, x.luz as luz_id
from [regional_forecast].[sr13_final].[mgrabase] x
join [regional_forecast].[sr13_final].[mgra13] y
on x.mgra = y.mgra
where scenario = 0'''
sr13_df = pd.read_sql(sr13_sql, mssql_engine)

#### get names of jurisdictions and cpas

In [ ]:
jurisdictions_names_sql = '''
    SELECT zone as jurisdiction_id, name as jurisdiction
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 150'''
jur_name_df = pd.read_sql(jurisdictions_names_sql, mssql_engine)
cocpa_names_sql = '''
    SELECT zone as cocpa_id, name as cocpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 20'''
cocpa_names = pd.read_sql(cocpa_names_sql, mssql_engine)
cicpa_names_sql = '''
    SELECT zone as cicpa_id, name as cicpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 15'''
cicpa_names = pd.read_sql(cicpa_names_sql, mssql_engine)
luz_names_sql = '''
    SELECT zone as luz_id, name as luz
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 64'''
luz_names = pd.read_sql(luz_names_sql, mssql_engine)     

#### merge results of sr13 and names

In [ ]:
sr13_df = pd.merge(sr13_df,jur_name_df[['jurisdiction_id','jurisdiction']],left_on='city',right_on='jurisdiction_id')
sr13_df = pd.merge(sr13_df,cocpa_names[['cocpa_id','cocpa']],left_on='cpa',right_on='cocpa_id', how = 'outer')
sr13_df = pd.merge(sr13_df,cicpa_names[['cicpa_id','cicpa']],left_on='cpa',right_on='cicpa_id',how = 'outer')
sr13_df = pd.merge(sr13_df,luz_names[['luz_id','luz']],left_on='luz_id',right_on='luz_id')
sr13_df['jur_or_cpa_name'] = sr13_df['cocpa']
sr13_df['jur_or_cpa_name'].fillna(sr13_df['cicpa'],inplace=True)
sr13_df['jur_or_cpa_name'].fillna(sr13_df['jurisdiction'],inplace=True)
sr13_df['jur_or_cpa_id'] = sr13_df['cocpa_id']
sr13_df['jur_or_cpa_id'].fillna(sr13_df['cicpa_id'],inplace=True)
sr13_df['jur_or_cpa_id'].fillna(sr13_df['jurisdiction_id'],inplace=True)
sr13_df['jur_or_cpa_id'] = sr13_df['jur_or_cpa_id'].astype(int)
sr13_df.drop(['jurisdiction_id', 'cocpa_id', 'cicpa_id', 'cocpa', 'cicpa'], axis=1,inplace=True)
# sr13_df.to_csv('test.csv')

In [ ]:
sr13_df.head()

## sr13 summary

In [ ]:
sr13summary = pd.DataFrame({'hs_sum': sr13_df.groupby(['increment']).
                               hs.sum()}).reset_index()
# sr13summary['hs_sum_shifted'] = sr13summary.hs_sum.shift(1)
sr13summary['chg'] = sr13summary.hs_sum.diff().fillna(0)
sr13summary['chg%'] = (sr13summary.hs_sum.pct_change().fillna(0) * 100).round(2)
# sr13summary.style.format({'chg%': '{:.2%}'})
sr13summary

# by jurisdiction

#### aggregrate mgra level housing to jurisdiction and group by increment and sum

In [ ]:
sr13_geo_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['jurisdiction','increment']).
                               hs.sum()}).reset_index()
sr13_geo_df.rename(columns = {'jurisdiction':'geo'},inplace=True)
sr13_geo_df.head()

In [ ]:
print("\nNum of geographies (jurisdictions) = {:,}\n".format(int(len(sr13_geo_df.loc[sr13_geo_df.increment==2015]))))

#### pivot so each jurisdiction is column and rows are increments

In [ ]:
sr13_geo_df_pivot = sr13_geo_df.pivot\
(index='increment', columns='geo', values='hs_sum').\
reset_index().rename_axis(None, axis=1)
sr13_geo_df_pivot.set_index('increment',inplace=True)
sr13_geo_df_pivot

#### calculate total change by region and jurisdiction

In [ ]:
total_diff = sr13_geo_df_pivot.loc[[2012,2050],:]
differences = total_diff.diff().loc[[2050]]
differences.rename(index={2050: 'total_change'},inplace=True)
totalchange = pd.DataFrame(differences.sum(axis=0))
totalchange.rename(columns={0: 'total_change'},inplace=True)
print("\nTotal Units added sr13: {:,}".format(int(totalchange.total_change.sum())))

In [ ]:
totalchange

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=2,figsize=(120, 40))
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
plt.xticks(rotation=70)
plt.subplots_adjust(top=0.25)
plt.suptitle("\nTotal Additional Housing Units: {:,}".format(int(totalchange.total_change.sum())),size=18)
for j, jur in enumerate(sr13_geo_df.geo.unique().tolist()):
    df_plot = sr13_geo_df_pivot[[jur]]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,30),x_compat=True)
    axes.flat[j].set_xlabel("Forecast Year",size=12)
    axes.flat[j].set_ylabel("Total Housing Units",size=12)
    axes.flat[j].yaxis.set_major_formatter(tick) 
    axes.flat[j].set_xticks([2012,2015,2020,2025,2030,2035,2040,2045,2050])
    axes.flat[j].set_xticklabels([2012,2015,2020,2025,2030,2035,2040,2045,2050],rotation=45)
    axes.flat[j].set_title('SERIES 13 REGIONAL GROWTH FORECAST',size=14) 
    axes.flat[j].legend(fontsize=14)
    handles, labels = axes.flat[j].get_legend_handles_labels()
    extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    labels.append('chg = ' + str(int(totalchange.loc[jur][0])))
    handles.append(extra)
    axes.flat[j].legend(handles, labels)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
# fig.savefig('sr13.png', format='png', dpi=300)


# by jurisdiction and cpa

#### aggregrate mgra level housing to cpa and jurisdiction and group by increment and sum

In [ ]:
sr13_geo_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['jur_or_cpa_name','increment']).
                               hs.sum()}).reset_index()
sr13_geo_df.rename(columns = {'jur_or_cpa_name':'geo'},inplace=True)
sr13_geo_df.head()

In [ ]:
print("\nNum of geographies (jurisdiction plus cpa) = {:,}\n".format(int(len(sr13_geo_df.loc[sr13_geo_df.increment==2015]))))

#### pivot so each jurisdiction or cpa is column and rows are increments

In [ ]:
sr13_geo_df_pivot = sr13_geo_df.pivot\
(index='increment', columns='geo', values='hs_sum').\
reset_index().rename_axis(None, axis=1)
sr13_geo_df_pivot.set_index('increment',inplace=True)
sr13_geo_df_pivot

#### calculate total change by region and jurisdiction and cpa

In [ ]:
total_diff = sr13_geo_df_pivot.loc[[2012,2050],:]
differences = total_diff.diff().loc[[2050]]
differences.rename(index={2050: 'total_change'},inplace=True)
totalchange = pd.DataFrame(differences.sum(axis=0))
totalchange.rename(columns={0: 'total_change'},inplace=True)
print("\nTotal Units added sr13: {:,}".format(int(totalchange.total_change.sum())))

In [ ]:
# totalchange

In [ ]:
sr13_geo_df_pivot

In [ ]:

pp = PdfPages("jur_and_cpa.pdf")
for j, jur in enumerate(sr13_geo_df.geo.unique().tolist()):
    # plt.figure()
    jur_and_cpa_plot = plt.figure()
    # plt.subplot(20, 1, j+1)
    plotlabel = jur + '\nchg = ' + str(int(totalchange.loc[jur][0]))
    plt.plot(sr13_geo_df_pivot[[jur]].reset_index().increment.tolist(),\
             sr13_geo_df_pivot[[jur]].reset_index()[jur].tolist(),\
             label = plotlabel)
    plt.legend()
    pp.savefig(jur_and_cpa_plot, dpi = 300, transparent = True)
pp.close()
# plt.savefig('sr13_jur_and_cpa.png')

# by luz

#### aggregrate mgra level housing to luz and group by increment and sum

In [ ]:
sr13_geo_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['luz','luz_id','increment']).
                               hs.sum()}).reset_index()
sr13_geo_df.rename(columns = {'luz':'geo'},inplace=True)
sr13_geo_df.sort_values(by='luz_id',inplace=True)
sr13_geo_df.head()

In [ ]:
print("\nNum of geographies (jurisdictions) = {:,}\n".format(int(len(sr13_geo_df.loc[sr13_geo_df.increment==2015]))))

#### pivot so each luz is column and rows are increments

In [ ]:
sr13_geo_df_pivot = sr13_geo_df.pivot\
(index='increment', columns='geo', values='hs_sum').\
reset_index().rename_axis(None, axis=1)
sr13_geo_df_pivot.set_index('increment',inplace=True)
sr13_geo_df_pivot

#### calculate total change by region and luz

In [ ]:
total_diff = sr13_geo_df_pivot.loc[[2012,2050],:]
differences = total_diff.diff().loc[[2050]]
differences.rename(index={2050: 'total_change'},inplace=True)
totalchange = pd.DataFrame(differences.sum(axis=0))
totalchange.rename(columns={0: 'total_change'},inplace=True)
print("\nTotal Units added sr13: {:,}".format(int(totalchange.total_change.sum())))

In [ ]:
# totalchange

In [ ]:
sr13_geo_df_pivot

In [ ]:
pp = PdfPages("luz.pdf")
for j, luz in enumerate(sr13_geo_df.geo.unique().tolist()):
    # plt.figure()
    jur_and_cpa_plot = plt.figure()
    # plt.subplot(20, 1, j+1)
    plotlabel = str(sr13_geo_df.loc[sr13_geo_df['geo']==luz].luz_id.values[0]) + '.' +\
                luz + '\nchg = ' + str(int(totalchange.loc[luz][0]))
    plt.plot(sr13_geo_df_pivot[[luz]].reset_index().increment.tolist(),\
             sr13_geo_df_pivot[[luz]].reset_index()[luz].tolist(),\
             label = plotlabel)
    plt.legend()
    pp.savefig(jur_and_cpa_plot, dpi = 300, transparent = True)
pp.close()
# plt.savefig('sr13_jur_and_cpa.png')